<a href="https://colab.research.google.com/github/Dhut/Myfifth/blob/master/Notebook/EP_S2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, to_categorical
from keras.preprocessing.sequence import pad_sequences
##for text cleaning
import re, string

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
!ls /content/drive/'My Drive'/

In [0]:
# load ascii text and covert to lowercase
filename = "Wonderland.txt"
raw_text = open("/content/drive/My Drive/" + filename).read()
raw_text = raw_text.lower()

In [4]:
print(raw_text[:500])

﻿project gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.org


title: alice's adventures in wonderland

author: lewis carroll

posting date: june 25, 2008 [ebook #11]
release date: march, 1994
[last updated: december 20, 2011]

language: e


# Remove Punctuations

In [5]:
raw_text = re.sub('[%s]' % re.escape(string.punctuation), '', raw_text)
print(raw_text[:500])

﻿project gutenbergs alices adventures in wonderland by lewis carroll

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever  you may copy it give it away or
reuse it under the terms of the project gutenberg license included
with this ebook or online at wwwgutenbergorg


title alices adventures in wonderland

author lewis carroll

posting date june 25 2008 ebook 11
release date march 1994
last updated december 20 2011

language english


 start of this


In [6]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters:", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters: 154861
Total Vocab:  39


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  154761


In [0]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

In [9]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))

W0721 16:08:41.630527 140636580714368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0721 16:08:41.646975 140636580714368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 16:08:41.650741 140636580714368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0721 16:08:41.995847 140636580714368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0721 16:08:42.003371 

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0718 14:26:40.301729 139818388506496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0718 14:26:40.337764 139818388506496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
# fit the model
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

Epoch 1/100
154761/154761 [==============================] - 487s 3ms/step - loss: 2.7189

Epoch 00001: loss improved from inf to 2.71888, saving model to weights-improvement-01-2.7189.hdf5
Epoch 2/100
154761/154761 [==============================] - 486s 3ms/step - loss: 2.3931

Epoch 00002: loss improved from 2.71888 to 2.39310, saving model to weights-improvement-02-2.3931.hdf5
Epoch 3/100
154761/154761 [==============================] - 488s 3ms/step - loss: 2.1796

Epoch 00003: loss improved from 2.39310 to 2.17956, saving model to weights-improvement-03-2.1796.hdf5
Epoch 4/100
154761/154761 [==============================] - 486s 3ms/step - loss: 2.0354

Epoch 00004: loss improved from 2.17956 to 2.03540, saving model to weights-improvement-04-2.0354.hdf5
Epoch 5/100
154761/154761 [==============================] - 487s 3ms/step - loss: 1.9327

Epoch 00005: loss improved from 2.03540 to 1.93268, saving model to weights-improvement-05-1.9327.hdf5
Epoch 6/100
154761/154761 [=======

In [0]:

# load the network weights
filename = "/content/drive/My Drive/weights-improvement-63-0.7781.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [19]:
# fit the model
model.fit(X, y, epochs=37, batch_size=128, callbacks=callbacks_list)

W0718 14:34:53.127633 139818388506496 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/37
154761/154761 [==============================] - 454s 3ms/step - loss: 0.7766

Epoch 00001: loss improved from inf to 0.77657, saving model to weights-improvement-01-0.7766.hdf5
Epoch 2/37
154761/154761 [==============================] - 453s 3ms/step - loss: 0.7608

Epoch 00002: loss improved from 0.77657 to 0.76079, saving model to weights-improvement-02-0.7608.hdf5
Epoch 3/37
154761/154761 [==============================] - 452s 3ms/step - loss: 0.7553

Epoch 00003: loss improved from 0.76079 to 0.75532, saving model to weights-improvement-03-0.7553.hdf5
Epoch 4/37
154761/154761 [==============================] - 451s 3ms/step - loss: 0.7610

Epoch 00004: loss did not improve from 0.75532
Epoch 5/37
154761/154761 [==============================] - 449s 3ms/step - loss: 0.7486

Epoch 00005: loss improved from 0.75532 to 0.74865, saving model to weights-improvement-05-0.7486.hdf5
Epoch 6/37
154761/154761 [==============================] - 450s 3ms/step - loss: 0.7416

Epoch

In [0]:

# load the network weights
filename = "/content/drive/My Drive/weights-improvement-36-0.6080.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [14]:
# fit the model
model.fit(X, y, epochs=3, batch_size=128, callbacks=callbacks_list)

W0721 16:12:22.870857 140636580714368 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/3
154761/154761 [==============================] - 325s 2ms/step - loss: 0.6132

Epoch 00001: loss improved from inf to 0.61318, saving model to weights-improvement-01-0.6132.hdf5
Epoch 2/3
154761/154761 [==============================] - 321s 2ms/step - loss: 0.6008

Epoch 00002: loss improved from 0.61318 to 0.60083, saving model to weights-improvement-02-0.6008.hdf5
Epoch 3/3
154761/154761 [==============================] - 320s 2ms/step - loss: 0.6003

Epoch 00003: loss improved from 0.60083 to 0.60027, saving model to weights-improvement-03-0.6003.hdf5


In [0]:
import sys

In [16]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" aging opening for a conversation alice replied
rather shyly ii hardly know sir just at presentat lea "
st to do with anowher minute theres no harge you she said to herself im any antthing blice thought she had never like that 
alice was that all telse the puppy filled with a little gound ier beotalne so do so alice they seemed to be a little lork coneusion that he said to the gryphon 
you meyer said the queen 
i wish i hadnt cried the mouse with a long and then
said together

what for said the king and the little door and the loral of that isce out why yould bll walted to see it tro this time and
Done.
